# PyTorch 中的循环神经网络模块
前面我们讲了循环神经网络的基础知识和网络结构，下面我们教大家如何在 pytorch 下构建循环神经网络，因为 pytorch 的动态图机制，使得循环神经网络非常方便。

## 一般的 RNN

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9xz889xj30kb07nglo.jpg)

对于最简单的 RNN，我们可以使用下面两种方式去调用，分别是 `torch.nn.RNNCell()` 和 `torch.nn.RNN()`，这两种方式的区别在于 `RNNCell()` 只能接受序列中单步的输入，且必须传入隐藏状态，而 `RNN()` 可以接受一个序列的输入，默认会传入全 0 的隐藏状态，也可以自己申明隐藏状态传入。

`RNN()` 里面的参数有

input_size 表示输入 $x_t$ 的特征维度

hidden_size 表示输出的特征维度

num_layers 表示网络的层数

nonlinearity 表示选用的非线性激活函数，默认是 'tanh'

bias 表示是否使用偏置，默认使用

batch_first 表示输入数据的形式，默认是 False，就是这样形式，(seq, batch, feature)，也就是将序列长度放在第一位，batch 放在第二位

dropout 表示是否在输出层应用 dropout

bidirectional 表示是否使用双向的 rnn，默认是 False

对于 `RNNCell()`，里面的参数就少很多，只有 input_size，hidden_size，bias 以及 nonlinearity

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
# 定义一个单步的rnn
rnn_single = nn.RNNCell(input_size=100, hidden_size=200)   # RNNCell()里面参数只有 input_size, hidden_size, bias, nonlinearity

In [3]:
# 访问其中的参数
rnn_single.weight_hh

Parameter containing:
tensor([[ 0.0372,  0.0532,  0.0556,  ...,  0.0182, -0.0300, -0.0476],
        [ 0.0641, -0.0329,  0.0089,  ..., -0.0439,  0.0679,  0.0170],
        [-0.0379, -0.0687,  0.0458,  ..., -0.0027,  0.0582,  0.0053],
        ...,
        [ 0.0498, -0.0198,  0.0464,  ...,  0.0130,  0.0651, -0.0542],
        [ 0.0397,  0.0049, -0.0477,  ...,  0.0263,  0.0463,  0.0191],
        [-0.0321, -0.0107,  0.0350,  ...,  0.0075, -0.0290,  0.0395]],
       requires_grad=True)

In [4]:
# 构造一序列，长为6， batch为5， 特征是100
x = Variable(torch.randn(6, 5, 100))     # 这是rnn的输入格式！记住就好
# 定义初始的记忆状态
h_t = Variable(torch.zeros(5, 200))

# 传入 rnn
out = []
for i in range(6):
    h_t = rnn_single(x[i], h_t)
    out.append(h_t)

In [6]:
len(out)

6

In [9]:
out[0].shape # 每个输出的维度

torch.Size([5, 200])

可以看到经过了 rnn 之后，隐藏状态的值已经被改变了，因为网络记忆了序列中的信息，同时输出 6 个结果

下面我们看看直接使用 `RNN`的情况

In [10]:
rnn_seq = nn.RNN(100, 200)

In [21]:
# 访问其中的参数
# rnn_seq.weight_hh_l0

out, h_t = rnn_seq(x) # 使用默认的全 0 隐藏状态

In [22]:
# h_t
len(out)

6

In [23]:
# 自己定义初始的隐藏状态
h_0 = Variable(torch.randn(1, 5, 200))

In [24]:
# 这里的隐藏状态的大小有三个维度，分别是 (num_layers * num_direction, batch, hidden_size)
out, h_t = rnn_seq(x, h_0)
out.shape                      # 输出为 （seq, batch, feature)

torch.Size([6, 5, 200])

一般情况下我们都是用 nn.RNN() 而不是 nn.RNNCell()，因为 nn.RNN() 能够避免我们手动写循环，非常方便，同时如果不特别说明，我们也会选择使用默认的全 0 初始化隐藏状态


## LSTM

![](https://ws1.sinaimg.cn/large/006tKfTcly1fmt9qj3uhmj30iz07ct90.jpg)

LSTM 和基本的 RNN 是一样的，他的参数也是相同的，同时他也有 `nn.LSTMCell()` 和 `nn.LSTM()` 两种形式，跟前面讲的都是相同的，我们就不再赘述了，下面直接举个小例子

In [28]:
lstm_seq = nn.LSTM(50, 100, num_layers=2)   # 输入维度100，输出维度200， 两层
# lstm_seq.weight_hh_10.shape
lstm_seq.weight_hh_l0 # 第一层的 h_t 权重

Parameter containing:
tensor([[ 0.0548, -0.0224,  0.0546,  ..., -0.0194,  0.0035,  0.0039],
        [ 0.0983,  0.0367,  0.0212,  ...,  0.0353,  0.0811, -0.0880],
        [-0.0187,  0.0015, -0.0970,  ..., -0.0562,  0.0408, -0.0796],
        ...,
        [-0.0719, -0.0961,  0.0613,  ..., -0.0242,  0.0252, -0.0661],
        [-0.0091, -0.0892,  0.0817,  ...,  0.0388, -0.0996, -0.0159],
        [ 0.0732,  0.0468,  0.0928,  ...,  0.0093, -0.0677, -0.0431]],
       requires_grad=True)

In [29]:
lstm_seq.weight_hh_l0.shape

torch.Size([400, 100])

In [30]:
lstm_input = Variable(torch.randn(10, 3, 50)) # 序列 10，batch 是 3，输入维度 50
out, (h, c) = lstm_seq(lstm_input) # 使用默认的全 0 隐藏状态

注意这里 LSTM 输出的隐藏状态有两个，h 和 c，就是上图中的每个 cell 之间的两个箭头，这两个隐藏状态的大小都是相同的，(num_layers * direction, batch, feature)

In [31]:
# 我们可以不使用默认的隐藏状态，这是需要传入两个张量
h_init = Variable(torch.randn(2, 3, 100))
c_init = Variable(torch.randn(2, 3, 100))

out, (h, c) = lstm_seq(lstm_input, (h_init, c_init))

In [32]:
print('h.shape',h.shape,'\t c.shape',c.shape,'\t out.shape',out.shape)

h.shape torch.Size([2, 3, 100]) 	 c.shape torch.Size([2, 3, 100]) 	 out.shape torch.Size([10, 3, 100])


# GRU
![](https://ws3.sinaimg.cn/large/006tKfTcly1fmtaj38y9sj30io06bmxc.jpg)

GRU 和前面讲的这两个是同样的道理，就不再细说，还是演示一下例子

In [33]:
gru_seq = nn.GRU(10, 20)
gru_input = Variable(torch.randn(3, 32, 10))

out, h = gru_seq(gru_input)

In [34]:
gru_seq.weight_hh_l0

Parameter containing:
tensor([[-0.1367, -0.1534,  0.1861,  ..., -0.0772,  0.1176,  0.0775],
        [ 0.1510, -0.0243,  0.1811,  ...,  0.0915,  0.1027,  0.0209],
        [ 0.1624,  0.1065,  0.1454,  ...,  0.0628, -0.1073, -0.1305],
        ...,
        [ 0.1948, -0.0937, -0.1434,  ...,  0.2168,  0.0837,  0.1831],
        [-0.0576,  0.0960, -0.1009,  ...,  0.1947, -0.0544, -0.1055],
        [-0.1230,  0.0755,  0.1718,  ..., -0.1473, -0.0738,  0.1418]],
       requires_grad=True)

In [35]:
print('h.shape',h.shape, '\t out.shape',out.shape)

h.shape torch.Size([1, 32, 20]) 	 out.shape torch.Size([3, 32, 20])
